In [57]:





while True:
    choice=int(input('0.exit \n 1.signin  \n 2. signup '))
    if choice==0:

        print('Succesfully Exited')

        break
    elif choice ==1:
       signin()
    elif choice ==2:
       signup_member()
       
    else:
        print('Wrong choice entered ! \n Please try again')

    


,id,user id,amount,type,transaction type,next payment due date
0,1,2,500,cash,2022-06-24 00:40:48.282834,None
1,2,2,500.0,cash,2022-06-24 00:45:07.968061,None
2,3,2,500.0,cash,2022-06-24 00:55:01.422094,None


Successfully Signed out
Succesfully Exited


In [26]:
def signin_librarian():
    while True:
        choice=int(input("1. edit profile \n2. change password \n3. find book by name \n4. check book availability \n5. add new book \n6. add new copy \n7. take payment and issue book copy  \n8. return book copy \n9. list issued books \n10. add member \n11. payment history \n12. list all users\n0. exit\n"))
        if choice==1:
            edit_profile()
            
        elif choice==2:
            change_password()
        elif choice==3:
            find_book()
        elif choice==4:
            check_book_availability()
        elif choice==5:
            add_new_book()
        elif choice==6:
            add_new_copy()
           
        
        elif choice==7:
            take_payment()
            issue_book_copy()
        elif choice==8:
            return_book_copy()
        elif choice==9:
            list_issued_books()
       
        
        elif choice==10:
            signup_member()
        
        
        elif choice==11:
            payment_history()
        elif choice==12:
            list_users()
        elif choice==0:
            print("Successfully exited !")
            break
        else:
            print('Wrong choice entered !!, Please try again')

            


    
    

In [28]:
def take_payment():
    global uid
    global e
    uid=int(input("Enter user id : "))
    e=int(input('Enter book_id : '))
    amt=float(input("Enter amount in Rupees:"))
    tp=input("Enter mode of transaction : ")
    import psycopg2
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute(''' select count(book_id) from copies where book_id=%s and status= 'available' ''',(e,) )
    r=cur.fetchall()
    if r[0][0]>0: 
        cur.execute('''insert into payments (user_id,amount,type)  values(%s,%s,%s)''',(uid,amt,tp))
    else: 
        print("Entered book id is unavailable")
    conn.commit()
    conn.close()

In [9]:
def list_users():
    import psycopg2
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''select name from user1''')
    print('Name of Users :')
    r=cur.fetchall()
    for i in r:
       print(i[0])
    conn.commit()
    conn.close()

In [10]:
list_users()

Name of Users :
amit
abu


In [29]:
signin_librarian()

Entered book id is unavailable
Successfully exited !


In [10]:
print(uid)
print(e)

2
10


In [102]:
def check_book_availability():
  name=input("Enter Book name : ")+'%'

  import psycopg2
  from psycopg2.extensions import AsIs
  conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
  cur=conn.cursor()
  cur.execute(''' select name,status from copies join books b on b.id=book_id where status='available' and book_id in (select id from books where name ilike '%s')''',(AsIs(name),))
  r=cur.fetchall()
  for i in range(len(r)):
    
    if r[i][1]=='available':
      print(r[i][0], '-->  available')

  if r==[]:
    print('not available')
    
  
  
  conn.commit()
  conn.close()

In [101]:
check_book_availability()

not available


In [5]:
def return_book_copy():
    re=input("Enter return date (in yyyy-mm-dd format) : ")
    mid=input("Enter member id : ")
    import psycopg2
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''update issuecard set return_date=%s where memberid=%s''', (re,mid))
    conn.commit()
    conn.close()

In [4]:
def list_issued_books():
    print('id | book_id | copyid |         name          | memberid | issuedate  | return_due_date | fine_amount')
    import psycopg2
    import datetime
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''select i.id,c.book_id,copyid,b.name,memberid,issuedate,return_due_date, ((return_date-return_due_date)/7)*50 as fine_amount from issuecard i join copies c on copyid=c.id join books b on book_id=b.id''')
    r=cur.fetchall()
    for i in r:
       print(i)
    conn.commit()
    conn.close()

In [5]:
list_issued_books()

id | book_id | copyid |         name          | memberid | issuedate  | return_due_date | fine_amount
(1, 7, 1, 'ms dhoni untold story', 2, datetime.date(2022, 6, 12), datetime.date(2022, 6, 19), None)
(3, 8, 4, 'harry potter', 1, datetime.date(2022, 6, 12), datetime.date(2022, 6, 19), 50)


In [17]:

def add_new_book():
    import psycopg2
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    print('Enter book information')
    a=input('Book name : ')
    b=input('Author name : ')
    c=input('Subject name : ')
    d=float(input('Price : '))
    e=int(input('ISBN : '))
	 
   
    cur.execute('''insert into books (name,author,subject,price,isbn) values(%s,%s,%s,%s,%s)''', (a,b,c,d,e))
    cur.execute('''insert into copies (book_id, rack) values((select * from currval('incc')), 1 )''')
    
    conn.commit()
    conn.close()
    print('Book added successfully')

In [2]:
add_new_copy()

Enter book information
Book added successfully


In [16]:
def add_new_copy():
    import psycopg2
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    x=int(input('Enter Book Id'))
    cur.execute('''insert into copies (book_id,rack) values(%s,1)''',x)
    conn.commit()
    conn.close()
    print('Copy added successfully')

In [15]:
def signup_member():
    import psycopg2
    from psycopg2.extensions import AsIs
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    name=(input('Enter name : '))
    email=(input('Enter email : '))
    phone=(input('Enter phone no. : '))
    pwd=(input('Enter password : '))
    cur.execute("""create user %s with password %s""", (AsIs(name),pwd,))
    cur.execute(''' grant select on books to %s ''', (AsIs(name),))
    cur.execute(''' grant all on user1 to %s ''', (AsIs(name),))
    cur.execute(''' insert into user1 (id,name,email,phone_no, password, role) values (nextval('user1_seq'),%s,%s,%s,%s,'member')''',(name,email,phone,pwd,) )

    conn.commit()
    conn.close()

In [21]:
signup_member()


In [50]:
def signin():
    global usr
    global pas
    usr=input('Enter name : ')
    pas=input('Enter password : ')
    
    import psycopg2
    conn=psycopg2.connect(database='library', user=usr, password=pas,host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute(''' select role from user1 where name=%s and password=%s''',(usr,pas))
    x=cur.fetchall()
    if x[0][0]=='member':
      signin_member()
        
        
    elif x[0][0]== 'librarian':
      signin_librarian()
    elif x[0][0]=='owner':
      signin_owner()
    conn.commit()
    

In [51]:

signin()
print(usr)

OperationalError: connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "amit"


In [7]:
def edit_profile():
    u=input('Enter new email : ')
    p=input('Enter new mobile_no. : ')

    import psycopg2
    
    conn=psycopg2.connect(database='library', user=usr, password=pas,host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''update user1 set email=%s , phone_no=%s where name=%s ''',(u,p,usr))
    conn.commit()
    conn.close()

In [8]:
def signin_owner():

   while True:
      choice=int(input('0. Signout \n 1. Edit profile  \n 2. Change password \n 3. Bookwise copies report \n 4. Fees/ Fine report '))
      if choice==0:  
          print('Successfully Signed out')   
          break
      elif choice ==1:
         edit_profile() 
      elif choice ==2:
         change_password() 
      
         
      elif choice ==3:
         bookwise_copies_report()
      elif choice ==4:
         payment_history() # ye bhi
      
         
      else:
          print('Wrong choice entered ! \n Please try again')

In [4]:
signin_owner()

BOOK ID  |  BOOK NAME  |  AVAILABLE  |  ISSUED  |  COUNT
(7, 'ms dhoni untold story', 2, 1, 3)
(8, 'harry potter', 2, 1, 3)
(9, 'avatar', 1, 2, 3)
(10, 'dunes', 1, 2, 3)
Successfully Signed out


In [47]:
def signin_member():
   while True:
      choice=int(input('0. Signout \n 1. Edit profile  \n 2. Change password \n 3. Find book by name \n 4. Check book availability \n 5. List issued books \n 6. Payment history '))
      if choice==0:
         print('Successfully Signed out')
         break
      elif choice ==1:
         edit_profile() 
         
      elif choice ==2:
         change_password()
      elif choice ==3:
         find_book()  
      elif choice ==4:
         check_book_avail() 
      elif choice ==5:
         list_issued_books() 
      elif choice ==6:
         payment_history_mem()
         
      else:
         print('Wrong choice entered ! \n Please try again')

In [49]:
usr

NameError: name 'usr' is not defined

In [10]:
def bookwise_copies_report():
    import psycopg2
    from psycopg2.extensions import AsIs
    conn=psycopg2.connect(database='library',user='postgres',password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    print("BOOK ID  |  BOOK NAME  |  AVAILABLE  |  ISSUED  |  COUNT")
    cur.execute('''  with two as (with one as (select b.id,name,count(c.status) as available from books b left join copies c on book_id=b.id where status='available'group by b.id) select o.id,name,available, count(status)-available as issued, count(status) count from copies,one o group by o.id,name,available,book_id ) select * from two group by id,name,available,issued,count order by id ''')
    r=cur.fetchall()
    for i in r:

        print(i)
    conn.close()

In [11]:
def change_password():
     pd=input('Enter new password : ')
    
     import psycopg2
     from psycopg2.extensions import AsIs
     conn=psycopg2.connect(database='library', user=usr, password=pas,host='127.0.0.1',port='5432')
     cur=conn.cursor()
     cur.execute('''alter role %s with password %s  ''',(AsIs(usr),pd))
     cur.execute('''update user1 set password =%s where name = %s ''',(pd,usr))
     conn.commit()
     conn.close()

In [12]:
def find_book():
    bk=input('Enter book name: ')+'%'

    import psycopg2
    from psycopg2.extensions import AsIs
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''select name,author,subject,price,isbn from books where name ilike '%s' ''',(AsIs(bk),))
    b=cur.fetchall()
    for i in b:

        print(i)
    
    conn.commit()
    conn.close()


In [16]:
bk=input('Enter book name: ')+'%'
bk

'ms%'

In [16]:
bookwise_copies_report()

(7, 'ms dhoni untold story', 2, 1, 3)
(8, 'harry potter', 2, 1, 3)
(9, 'avatar', 1, 2, 3)
(10, 'dunes', 1, 2, 3)


In [13]:
def check_book_avail():
    
    import psycopg2
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    
    print("BOOK ID  |  BOOK NAME  |  RACK  |  STATUS")
    cur.execute('''select c.id,name,rack, status from copies c left join books b on b.id=book_id;''')
    b=cur.fetchall()
    for i in b:

        print(i)
    
    conn.commit()
    conn.close()


In [40]:
import psycopg2
conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
cur=conn.cursor()
cur.execute('''SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'copies' ''')
b=cur.fetchall()
df = pd.DataFrame(tupples, columns=column_names)
c=[]
for i in b:
    c.extend(i)
print(c)
conn.commit()
conn.close()

['id', 'book_id', 'rack', 'status']


In [21]:
def issue_book_copy():
   
    import psycopg2
    
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    cur.execute('''insert into issuecard (copyid,memberid) values ((select id from copies where book_id=%s and status='available' order by id limit 1),%s)''',(e,uid))
    cur.execute('''with one as (select id from copies where book_id=%s and status='available' order by id limit 1) update copies set status='not available' where id=(select id from one)''',(e,))
    conn.commit()
    conn.close()

In [20]:
issue_book_copy()

In [21]:

    import psycopg2
    import pandas as pd
    
    column_names = ["id", "name", "email", "phone_no", "password" ,"role"]
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    
    cur.execute('''select*from user1''')
    tupples=cur.fetchall()
    df = pd.DataFrame(tupples, columns=column_names)
    
    
    print(df)
    conn.commit()
    conn.close()
   

   id  name          email   phone_no password    role
0   1  amit       amit.com    6546532     sing  member
1   2   abu  abu@gmail.com  981495654   shaikh  member


In [41]:
def payment_history():

    import psycopg2
    import pandas as pd
    
    column_names = ["id", "user id", "amount", "type", "transaction type" ,"next payment due date"]
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    
    cur.execute('''select*from payments''')
    tupples=cur.fetchall()
    df = pd.DataFrame(tupples, columns=column_names)
    

    
    display(df)

In [42]:
payment_history()

,id,user id,amount,type,transaction type,next payment due date
0,1,2,500,cash,2022-06-24 00:40:48.282834,None
1,2,2,500.0,cash,2022-06-24 00:45:07.968061,None
2,3,2,500.0,cash,2022-06-24 00:55:01.422094,None


In [56]:
def payment_history_mem():
    import psycopg2
    import pandas as pd
    
    column_names = ["id", "user id", "amount", "type", "transaction type" ,"next payment due date"]
    conn=psycopg2.connect(database='library', user='postgres', password='pgdbda',host='127.0.0.1',port='5432')
    cur=conn.cursor()
    
    cur.execute('''select*from payments where user_id=(select id from user1 where name=%s and password=%s)''',(usr,pas))
    tupples=cur.fetchall()
    df = pd.DataFrame(tupples, columns=column_names)
    

    
    display(df)
    conn.commit()
    conn.close()

In [48]:
payment_history_mem()

NameError: name 'usr' is not defined